In [4]:
!git clone https://github.com/katulevskiy/NLP-OpenBookQA.git

fatal: destination path 'NLP-OpenBookQA' already exists and is not an empty directory.


In [18]:
'''
Imports
'''

from sentence_transformers import SentenceTransformer, util
import torch

In [15]:
'''
Load model + list of facts, get fact embeddings. for now just using all-mpbnet-base-v2
'''

model_name = "all-mpnet-base-v2"
model = SentenceTransformer(model_name)

facts = []
facts_file = open("/content/NLP-OpenBookQA/OpenBookQA-V1-Sep2018/Data/Main/openbook.txt","r")
for fact in facts_file.readlines():
  facts.append(fact[1:len(fact)-2])

fact_embeddings = model.encode(facts, convert_to_tensor=True)

In [19]:
torch.save(fact_embeddings,"fact_embeddings.pt")

In [74]:
def get_top_k_facts(query,embedding_model,fact_embeddings,k=3, per_answer=False):
  if not per_answer:
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, fact_embeddings)[0]
    top_k_indices = cos_scores.topk(k=k).indices
    results = [facts[idx] for idx in top_k_indices]
    return results

  queries = []
  question = query.split('(A)')[0][:-1]

  answer_a = query.split('(A)')[1].split('(B)')[0][:-1]
  queries.append(question + answer_a)
  answer_b = query.split('(B)')[1].split('(C)')[0][:-1]
  queries.append(question + answer_b)
  answer_c = query.split('(C)')[1].split('(D)')[0][:-1]
  queries.append(question + answer_c)
  answer_d = query.split('(D)')[1]
  queries.append(question + answer_d)

  results = []
  for query in queries:
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, fact_embeddings)[0]
    top_k_indices = cos_scores.topk(k=k).indices
    for idx in top_k_indices:
      results.append(facts[idx])
      fact_embeddings = torch.cat((fact_embeddings[:idx], fact_embeddings[idx+1:]))

  return results

In [72]:
fact_embeddings = torch.load("fact_embeddings.pt",weights_only=True)

query = "The sun is responsible for	(A) puppies learning new tricks (B) children growing up and getting old (C) flowers wilting in a vase (D) plants sprouting, blooming and wilting"

print("Getting top 3 for each answer choice:\n")
for fact in get_top_k_facts(query,model,fact_embeddings,per_answer=True):
  print(fact)

print("\n\n")
print("Getting top 12 for query with all answer choices:\n")
for fact in get_top_k_facts(query,model,fact_embeddings,k=12):
  print(fact)


Getting top 3 for each answer choice:

the sun is the source of energy for life on Earth
the sun is the source of solar energy called sunlight
the sun is a source of light called sunlight
the sun is located directly overhead at noon
the sun is the source of energy for life on Earth
the sun is a source of heat called sunlight
a plant requires sunlight to grow
a plant requires sunlight for photosynthesis
if a tree falls then sunlight becomes available to the surrounding plants
a leaf absorbs sunlight to perform photosynthesis
photosynthesis means green plants convert from carbon dioxide, water, and solar energy into oxygen for themselves
a plant light is used for help plants by mimicking sunlight



Getting top 12 for query with all answer choices:

the sun is the source of energy for life on Earth
the sun is the source of energy for physical cycles on Earth
the sun is the source of solar energy called sunlight
the sun is a source of light called sunlight
the sun is a source of heat call

In [73]:
query = "When standing miles away from Mount Rushmore (A) the mountains seem very close (B) the mountains are boring (C) the mountains look the same as from up close (D) the mountains seem smaller than in photographs"

print("Getting top 3 for each answer choice:\n")
for fact in get_top_k_facts(query,model,fact_embeddings,per_answer=True):
  print(fact)

print("\n\n")
print("Getting top 12 for query with all answer choices:\n")
for fact in get_top_k_facts(query,model,fact_embeddings,k=12):
  print(fact)

Getting top 3 for each answer choice:

as distance from an object decreases , that object will appear larger
the surface of the Earth contains mountains
if an object is close then that object will appear large
the sun transfers heat energy from itself to the planets through sunlight
motion is a source of kinetic energy in an object
landfills have a negative impact on the communities
telephones convert sound energy into electrical energy
if an object is blue then that object reflects only blue light
as distance to a city decreases , the amount of light pollution will increase
as the size of a leaf increases , the amount of sunlight absorbed by that leaf will increase
as the size of a flower increases , the number of pollinators it will attract increases
loss of resources has a negative impact on the organisms in an area



Getting top 12 for query with all answer choices:

the surface of the Earth contains mountains
the surface of the Moon contains mountains
as distance from an object d